# ساخت عوامل هوش مصنوعی با حافظه پایدار با استفاده از Cognee

این دفترچه نشان می‌دهد که چگونه می‌توان عوامل هوش مصنوعی هوشمند با قابلیت‌های پیشرفته حافظه را با استفاده از [**cognee**](https://www.cognee.ai/) - یک حافظه هوش مصنوعی متن‌باز که گراف‌های دانش، جستجوی معنایی و مدیریت جلسات را ترکیب می‌کند تا سیستم‌های هوش مصنوعی آگاه به زمینه ایجاد کند، ساخت.

## 🎯 اهداف یادگیری

تا پایان این آموزش، شما خواهید آموخت که چگونه:
- **ساخت گراف‌های دانش مبتنی بر تعبیه‌ها**: تبدیل متن غیرساختاریافته به دانش ساختاریافته و قابل پرس‌وجو
- **پیاده‌سازی حافظه جلسه**: ایجاد مکالمات چند مرحله‌ای با حفظ خودکار زمینه
- **ذخیره مکالمات**: به صورت اختیاری تعاملات مهم را در حافظه بلندمدت برای استفاده در آینده ذخیره کنید
- **پرس‌وجو با استفاده از زبان طبیعی**: دسترسی و استفاده از زمینه تاریخی در مکالمات جدید
- **بصری‌سازی حافظه**: بررسی روابط در گراف دانش عامل خود


## 🏗️ آنچه خواهید ساخت

در این آموزش، یک **دستیار کدنویسی** با حافظه پایدار ایجاد خواهیم کرد که:

### 1. **ساخت پایگاه دانش**
   - اطلاعات پروفایل و تخصص توسعه‌دهنده را دریافت می‌کند
   - اصول و بهترین شیوه‌های برنامه‌نویسی پایتون را پردازش می‌کند
   - مکالمات تاریخی بین توسعه‌دهندگان و دستیارهای هوش مصنوعی را ذخیره می‌کند

### 2. **مکالمات آگاه از جلسه**
   - زمینه را در طول چندین سؤال در یک جلسه حفظ می‌کند
   - هر جفت سؤال/پاسخ را به‌طور خودکار برای بازیابی کارآمد ذخیره می‌کند
   - پاسخ‌های منسجم و مرتبط بر اساس تاریخچه مکالمه ارائه می‌دهد

### 3. **حافظه بلندمدت**
   - مکالمات مهم را در حافظه بلندمدت ذخیره می‌کند
   - خاطرات مرتبط را از پایگاه دانش و جلسات گذشته برای اطلاع‌رسانی تعاملات جدید بازیابی می‌کند
   - پایگاه دانشی ایجاد می‌کند که با گذشت زمان بهبود می‌یابد

### 4. **بازیابی هوشمند حافظه**
   - از جستجوی معنایی مبتنی بر گراف برای یافتن اطلاعات مرتبط در میان تمام دانش ذخیره‌شده استفاده می‌کند
   - جستجوها را بر اساس زیرگروه‌های داده (اطلاعات توسعه‌دهنده در مقابل اصول) فیلتر می‌کند
   - منابع داده مختلف را ترکیب می‌کند تا پاسخ‌های جامع ارائه دهد


## 📋 پیش‌نیازها و تنظیمات

### الزامات سیستم

قبل از شروع، مطمئن شوید که موارد زیر را دارید:

1. **محیط پایتون**
   - پایتون نسخه 3.9 یا بالاتر
   - استفاده از محیط مجازی (توصیه می‌شود)

2. **کش Redis** (ضروری برای مدیریت جلسات)
   - Redis محلی: `docker run -d -p 6379:6379 redis`
   - یا استفاده از یک سرویس مدیریت‌شده Redis

3. **دسترسی به API مدل‌های زبانی**
   - کلید API OpenAI یا سایر ارائه‌دهندگان (به [مستندات](https://docs.cognee.ai/setup-configuration/llm-providers) مراجعه کنید)

4. **پیکربندی پایگاه داده**
   - به‌طور پیش‌فرض نیازی به پیکربندی نیست. Cognee از پایگاه داده‌های مبتنی بر فایل (LanceDB و Kuzu) استفاده می‌کند.
   - به‌صورت اختیاری، می‌توانید Azure AI Search را به‌عنوان یک ذخیره‌ساز برداری تنظیم کنید (به [مستندات](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch) مراجعه کنید).

### پیکربندی محیط

یک فایل `.env` در دایرکتوری پروژه خود ایجاد کنید و متغیرهای زیر را وارد کنید:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ درک معماری حافظه Cognee

### نحوه عملکرد Cognee

Cognee یک سیستم حافظه پیشرفته ارائه می‌دهد که فراتر از ذخیره‌سازی ساده کلید-مقدار عمل می‌کند:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### اجزای کلیدی:

1. **گراف دانش**: ذخیره موجودیت‌ها، روابط و ارتباطات معنایی
2. **بردارهای جاسازی**: امکان جستجوی معنایی در تمام اطلاعات ذخیره‌شده
3. **کش جلسه**: حفظ زمینه مکالمه در جلسات و بین جلسات
4. **NodeSets**: سازماندهی داده‌ها در دسته‌های منطقی برای بازیابی هدفمند

### انواع حافظه در این آموزش:

- **حافظه پایدار**: ذخیره‌سازی بلندمدت در گراف دانش
- **حافظه جلسه**: زمینه مکالمه موقت در کش Redis
- **حافظه معنایی**: جستجوی شباهت مبتنی بر بردار در تمام داده‌ها


## 📦 نصب بسته‌های مورد نیاز

Cognee را با پشتیبانی Redis برای مدیریت جلسات نصب کنید:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 راه‌اندازی محیط و بارگذاری کتابخانه‌ها

اطمینان حاصل کنید:
1. Redis در حال اجرا باشد (مثلاً با استفاده از Docker: `docker run -d -p 6379:6379 redis`)
2. متغیرهای محیطی قبل از وارد کردن ماژول‌های کش تنظیم شده باشند
3. در صورت نیاز، کرنل را مجدداً راه‌اندازی کرده و سلول‌ها را به ترتیب اجرا کنید

سلول زیر:
1. متغیرهای محیطی را از فایل `.env` بارگذاری می‌کند
2. Cognee را با تنظیمات LLM شما پیکربندی می‌کند
3. کش را برای مدیریت جلسه فعال می‌کند
4. بررسی می‌کند که همه اجزا به درستی متصل شده باشند


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 تنظیم دایرکتوری‌های ذخیره‌سازی

Cognee از دو دایرکتوری جداگانه برای عملیات خود استفاده می‌کند:
- **ریشه داده**: ذخیره اسناد وارد شده و داده‌های پردازش شده
- **ریشه سیستم**: شامل پایگاه داده گراف دانش و متادیتای سیستم

ما برای این آموزش دایرکتوری‌های جداگانه‌ای به صورت زیر ایجاد خواهیم کرد:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 تنظیم مجدد حالت حافظه

قبل از اینکه سیستم حافظه خود را بسازیم، بیایید مطمئن شویم که از ابتدا شروع می‌کنیم.

> 💡 **نکته**: اگر می‌خواهید حافظه‌های موجود از اجراهای قبلی خود را هنگام استفاده مجدد از این دفترچه حفظ کنید، می‌توانید این مرحله را رد کنید.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 بخش ۱: ساخت پایگاه دانش

### منابع داده برای دستیار توسعه‌دهنده ما

ما سه نوع داده را برای ایجاد یک پایگاه دانش جامع وارد می‌کنیم:

1. **پروفایل توسعه‌دهنده**: تخصص شخصی و پیشینه فنی
2. **بهترین روش‌های پایتون**: اصول پایتون همراه با راهنمایی‌های عملی
3. **مکالمات تاریخی**: جلسات پرسش و پاسخ گذشته بین توسعه‌دهندگان و دستیارهای هوش مصنوعی

این داده‌های متنوع به عامل ما اجازه می‌دهد:
- زمینه فنی کاربر را درک کند
- بهترین روش‌ها را در توصیه‌ها اعمال کند
- از تعاملات موفق قبلی یاد بگیرد


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 تبدیل داده‌ها به نمودار دانش

اکنون متن خام خود را به یک حافظه ساختاریافته تبدیل می‌کنیم. این فرآیند:

1. **اضافه کردن داده‌ها به NodeSets**: سازماندهی اطلاعات در دسته‌بندی‌های منطقی
   - `developer_data`: پروفایل توسعه‌دهنده و مکالمات
   - `principles_data`: بهترین روش‌ها و دستورالعمل‌های پایتون

2. **اجرای Cognify Pipeline**: استخراج موجودیت‌ها، روابط و ایجاد تعبیه‌ها
   - شناسایی مفاهیم کلیدی
   - ایجاد ارتباطات معنایی بین اطلاعات مرتبط
   - تولید تعبیه‌های برداری

این ممکن است چند لحظه طول بکشد زیرا LLM متن را پردازش کرده و ساختار نمودار را ایجاد می‌کند:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 نمایش گراف دانش

بیایید ساختار گراف دانش خود را بررسی کنیم. این نمایش شامل موارد زیر است:
- **گره‌ها**: موجودیت‌هایی که از متن استخراج شده‌اند (مفاهیم، فناوری‌ها، افراد)
- **پیوندها**: روابط و ارتباطات بین موجودیت‌ها
- **خوشه‌ها**: مفاهیم مرتبط که بر اساس شباهت معنایی گروه‌بندی شده‌اند

فایل HTML تولید شده را در مرورگر خود باز کنید تا بتوانید به صورت تعاملی گراف را بررسی کنید:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 تقویت حافظه با Memify

تابع `memify()` نمودار دانش را تحلیل کرده و قوانین هوشمندی درباره داده‌ها ایجاد می‌کند. این فرآیند:
- الگوها و بهترین روش‌ها را شناسایی می‌کند
- دستورالعمل‌های قابل اجرا بر اساس محتوا ایجاد می‌کند
- روابط بین حوزه‌های مختلف دانش را برقرار می‌کند

این قوانین به عامل کمک می‌کنند تا هنگام پاسخ به سوالات تصمیمات آگاهانه‌تری بگیرد. ثبت یک تصویر دوم از نمودار به شما امکان می‌دهد مقایسه کنید که چگونه نمودار پس از غنی‌سازی متراکم‌تر می‌شود.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 بخش ۲: بازیابی هوشمند حافظه

### نمایش ۱: یکپارچه‌سازی دانش میان اسناد

حالا که گراف دانش ما ساخته شده است، بیایید آزمایش کنیم که چگونه Cognee اطلاعات را از منابع مختلف ترکیب می‌کند تا به سوالات پیچیده پاسخ دهد.

اولین پرسش نشان می‌دهد:
- **درک معنایی**: یافتن مفاهیم مرتبط حتی زمانی که به‌طور صریح ذکر نشده‌اند
- **ارجاع متقابل**: ترکیب پروفایل توسعه‌دهنده با اصول Python
- **استدلال زمینه‌ای**: اعمال بهترین روش‌ها در پیاده‌سازی‌های خاص

### نمایش ۲: جستجوی فیلتر شده با NodeSets

پرسش دوم نشان می‌دهد که چگونه می‌توان بخش‌های خاصی از گراف دانش را هدف قرار داد:
- از پارامتر `node_name` برای جستجو فقط در `principles_data` استفاده می‌کند
- پاسخ‌های متمرکز از یک حوزه دانش خاص ارائه می‌دهد
- مفید برای زمانی که به اطلاعات خاص حوزه نیاز دارید


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 بخش ۳: تنظیم مدیریت جلسه

### فعال‌سازی حافظه مکالمه

مدیریت جلسه برای حفظ زمینه در تعاملات متعدد بسیار مهم است. در اینجا ما:

1. **ایجاد زمینه کاربر**: یک پروفایل کاربر ایجاد کنید یا بازیابی کنید برای ردیابی جلسه
2. **پیکربندی موتور کش**: اتصال به Redis برای ذخیره تاریخچه مکالمات
3. **فعال‌سازی متغیرهای جلسه**: تنظیم متغیرهای زمینه که در پرسش‌ها باقی می‌مانند

> ⚠️ **مهم**: این نیاز به اجرای Redis و `CACHING=true` در محیط شما دارد


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ تابع کمکی: مشاهده تاریخچه جلسه

این تابع کمکی به ما امکان می‌دهد تاریخچه مکالمات ذخیره‌شده در Redis را بررسی کنیم. این تابع برای موارد زیر مفید است:
- رفع اشکال مدیریت جلسات
- اطمینان از اینکه مکالمات ذخیره می‌شوند
- درک اینکه چه زمینه‌ای برای عامل موجود است


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## جلسه ۱: آزمایش پشتیبانی غیرهمزمان — سوال اول

جلسه `async-support-lab` را با درخواست الگوهای مناسب و سازگار با تله‌متری برای asyncio در یک وب‌اسکرپر بزرگ آغاز کنید. گراف از قبل با asyncio، aiohttp و روش‌های نظارت آشنا است، بنابراین پاسخ باید مشابه مکالمات قبلی باشد و در عین حال به پرسش جدید اختصاص داده شود.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## بررسی حافظه جلسه ۱ پس از اولین تبادل

اجرای `show_history(session_1)` بلافاصله پس از پرسش اولیه تأیید می‌کند که Cognee هم پرسش و هم پاسخ را در Redis ذخیره کرده است. شما باید یک ورودی با راهنمایی همزمانی مشاهده کنید.


In [ ]:
await show_history(session_1)

## جلسه ۱: پیگیری مدل‌های داده

در ادامه می‌پرسیم: "چه زمانی باید dataclasses را انتخاب کنم و چه زمانی Pydantic را؟" با استفاده از همان شناسه جلسه. Cognee باید اصول Python و مکالمات قبلی FastAPI را ترکیب کند تا مشاوره‌ای دقیق ارائه دهد—نشان دادن اینکه زمینه در یک جلسه نام‌گذاری شده منتقل می‌شود.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## تأیید کنید که تاریخچه جلسه ۱ شامل هر دو نوبت است

یک بار دیگر فراخوانی `show_history(session_1)` باید دو ورودی پرسش و پاسخ را نشان دهد. این با مرحله "بازپخش حافظه" آزمایشگاه Mem0 مطابقت دارد و ثابت می‌کند که نوبت‌های اضافی همان متن را گسترش می‌دهند.


In [ ]:
await show_history(session_1)

## جلسه ۲: رشته بررسی طراحی — جلسه جدید

برای نشان دادن جداسازی بین رشته‌ها، `design-review-session` را راه‌اندازی می‌کنیم و برای بررسی حوادث راهنمایی ثبت‌گزارش درخواست می‌کنیم. حتی با وجود اینکه پایگاه دانش اصلی یکسان است، شناسه جلسه جدید متن‌ها را جدا نگه می‌دارد.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## بررسی جلسه ۲ تاریخچه

`show_history(session_2)` باید فقط جفت درخواست/پاسخ بررسی طراحی را فهرست کند. آن را با جلسه ۱ مقایسه کنید تا نشان دهید چگونه Cognee متن‌های مستقل را حفظ می‌کند در حالی که از نمودار دانش مشترک استفاده می‌کند.


In [ ]:
await show_history(session_2)

## خلاصه

تبریک می‌گوییم! شما به دستیار کدنویسی خود یک لایه حافظه بلندمدت واقعی با قدرت Cognee اضافه کردید.

در این آموزش، محتوای خام توسعه‌دهنده (کد، مستندات، چت‌ها) را به یک گراف + حافظه برداری تبدیل کردید که عامل شما می‌تواند در آن جستجو کند، استدلال کند و به طور مداوم بهبود یابد.

آنچه آموختید:

1. **از متن خام تا حافظه هوش مصنوعی**: نحوه پردازش داده‌های غیرساختاری توسط Cognee و تبدیل آن‌ها به حافظه هوشمند و قابل جستجو با استفاده از معماری ترکیبی گراف دانش + بردار.

2. **غنی‌سازی گراف با memify**: نحوه فراتر رفتن از ایجاد گراف پایه و استفاده از memify برای افزودن حقایق مشتق‌شده و روابط غنی‌تر به گراف موجود.

3. **استراتژی‌های مختلف جستجو**: نحوه پرس‌وجو از حافظه با انواع مختلف جستجو (پرسش و پاسخ مبتنی بر گراف، تکمیل به سبک RAG، بینش‌ها، قطعات خام، جستجوی کد و غیره) بسته به نیاز عامل شما.

4. **کاوش بصری**: نحوه بررسی و رفع اشکال آنچه Cognee ساخته است با استفاده از تصویری‌سازی گراف و رابط کاربری Cognee، تا بتوانید ساختار دانش را به صورت واقعی مشاهده کنید.

5. **حافظه آگاه از جلسه**: نحوه ترکیب زمینه هر جلسه با حافظه معنایی پایدار تا عوامل بتوانند بین اجراها اطلاعات را به خاطر بسپارند بدون اینکه اطلاعات بین کاربران نشت کند.


## نکات کلیدی
1. حافظه به‌عنوان یک گراف دانش پشتیبانی‌شده توسط Embeddings

   - **درک ساختاری**: Cognee ترکیبی از یک ذخیره‌سازی برداری و یک ذخیره‌سازی گراف است، بنابراین داده‌های شما هم بر اساس معنا قابل جستجو هستند و هم از طریق روابط به هم متصل‌اند. به‌طور پیش‌فرض، Cognee از پایگاه‌های داده مبتنی بر فایل استفاده می‌کند (LanceDB برای ذخیره‌سازی برداری، Kuzu برای پایگاه داده گراف).

   - **بازیابی آگاه به روابط**: پاسخ‌ها نه تنها بر اساس "متن مشابه"، بلکه بر اساس نحوه ارتباط موجودیت‌ها نیز قابل ارائه هستند.

   - **حافظه زنده**: لایه حافظه تکامل می‌یابد، رشد می‌کند و به‌عنوان یک گراف متصل قابل جستجو باقی می‌ماند.

2. حالت‌های جستجو و استدلال
   - **بازیابی ترکیبی**: جستجو ترکیبی از شباهت برداری، ساختار گراف و استدلال LLM است، از جستجوی خام تا پاسخ‌دهی به سوالات آگاه به گراف.

   - **تناسب حالت با وظیفه**: از حالت‌های تکمیل‌محور زمانی استفاده کنید که پاسخ‌های زبان طبیعی می‌خواهید، و از حالت‌های chunk/خلاصه/گراف زمانی که عامل شما به زمینه خام یا هدایت استدلال خود نیاز دارد.

3. عوامل شخصی‌سازی‌شده و آگاه به جلسه
   - **زمینه جلسه + حافظه بلندمدت**: Cognee زمینه کوتاه‌مدت "رشته" را جدا از حافظه بلندمدت در سطح کاربر یا سازمان نگه می‌دارد.

## کاربردهای واقعی

1. **عوامل هوش مصنوعی عمودی**

   از الگوی این دفترچه یادداشت برای قدرت‌بخشی به دستیاران هوشمند حوزه‌ای که بر اساس Cognee به‌عنوان هسته بازیابی و استدلال خود عمل می‌کنند، استفاده کنید:

- **دستیاران توسعه‌دهنده**: بررسی کد، تحلیل حادثه، و دستیاران معماری که کد، APIها، مستندات طراحی، و تیکت‌ها را به‌عنوان یک گراف حافظه واحد مرور می‌کنند.

- **دستیاران مشتری‌محور**: عوامل پشتیبانی یا موفقیت که از مستندات محصول، سوالات متداول، یادداشت‌های CRM، و تیکت‌های گذشته با بازیابی آگاه به گراف و پاسخ‌های مستند استفاده می‌کنند.

- **دستیاران متخصص داخلی**: دستیاران سیاست، حقوقی، یا امنیتی که بر اساس قوانین، دستورالعمل‌ها، و تصمیمات تاریخی مرتبط استدلال می‌کنند، نه فایل‌های PDF جداگانه.

   Cognee به‌طور خاص به‌عنوان حافظه پایدار و دقیق برای عوامل هوش مصنوعی موقعیت‌یابی شده است، و یک گراف دانش زنده ارائه می‌دهد که پشت عامل شما قرار می‌گیرد و جایگزین ترکیب‌های موقت ذخیره‌سازی برداری و کد گراف سفارشی می‌شود.

2. **یکپارچه‌سازی سیلوهای داده به یک حافظه**

   همین رویکرد به شما کمک می‌کند یک لایه حافظه یکپارچه در میان منابع پراکنده بسازید:

- **از سیلوها به یک گراف**: داده‌های ساختاریافته (مانند پایگاه‌های داده) و غیرساختاریافته (مانند مستندات، چت‌ها) را به یک گراف واحد پشتیبانی‌شده توسط Embeddings وارد کنید، به‌جای شاخص‌های جداگانه برای هر سیستم.

- **استدلال بین‌منبعی با استناد**: استدلال چندمرحله‌ای را بر روی همه چیز اجرا کنید—"اتصال" لاگ‌ها، معیارها، و مستندات از طریق گراف—و همچنان پاسخ‌های مستند با منبع ارائه دهید.

- **هاب‌های دانش**: برای حوزه‌هایی مانند بانکداری یا آموزش، Cognee قبلاً برای یکپارچه‌سازی فایل‌های PDF، سیستم‌های داخلی، و داده‌های اپلیکیشن به یک گراف دانش با بردارها استفاده شده است، تا عوامل بتوانند سوالات را با زمینه دقیق و مستند پاسخ دهند.

## مراحل بعدی

شما حلقه حافظه اصلی را پیاده‌سازی کرده‌اید. در اینجا گسترش‌های طبیعی وجود دارد که می‌توانید خودتان امتحان کنید (برای جزئیات به [مستندات Cognee](https://docs.cognee.ai/) مراجعه کنید):

1. **آزمایش با آگاهی زمانی**: قابلیت cognify زمانی را فعال کنید تا رویدادها و زمان‌ها را از متن استخراج کنید.

2. **معرفی استدلال مبتنی بر هستی‌شناسی**: یک هستی‌شناسی OWL برای حوزه خود تعریف کنید. از پشتیبانی هستی‌شناسی Cognee استفاده کنید تا موجودیت‌ها و روابط استخراج‌شده در آن طرح‌واره قرار گیرند، و کیفیت گراف و پاسخ‌های خاص حوزه را بهبود دهید.

3. **افزودن حلقه بازخورد**: اجازه دهید Cognee وزن لبه‌های گراف را از بازخورد واقعی کاربران تنظیم کند، تا بازیابی به‌مرور زمان بهبود یابد و ثابت نماند.

4. **تنظیم برای شخصی‌سازی و رفتار جلسه**: از شناسه‌های کاربر، مستاجرها، و مجموعه داده‌ها استفاده کنید تا هر فرد یا تیم نمای خود را بر روی موتور حافظه مشترک داشته باشد.

5. **گسترش به عوامل پیچیده‌تر**: Cognee را به چارچوب‌های عامل متصل کنید تا سیستم‌های چندعاملی بسازید که همه از یک لایه حافظه مشترک استفاده کنند. *پلاگین Microsoft Agent Framework x Cognee به‌زودی ارائه می‌شود.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حیاتی، ترجمه حرفه‌ای انسانی توصیه می‌شود. ما مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
